In [ ]:
from modules.data import load_dataset , view_random_images , augment_dataset , get_split_stats , create_stratified_split
from modules.preprocessing import preprocess_image
from modules.feature_extraction import extract_features , extract_features_skimage
from modules.models import nn_model , svm_model , save_model , load_model

import numpy as np

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# stored variables are:
# %store -r images
# %store -r labels
# %store -r augmented_images
# %store -r augmented_labels
# %store -r train_images
# %store -r train_labels
# %store -r test_images
# %store -r test_labels
# %store -r val_images
# %store -r val_labels


%store -r

In [ ]:
#Load the dataset and view some random images

# images , labels = load_dataset('dataset')
# %store images
# %store labels

print("DataSet Size :" , len(images))
assert len(images) == len(labels) , "Images and labels are not of the same size"
view_random_images(images, labels , num_images=10)

In [ ]:
#Augment the dataset and view some random images

# augmented_images , augmented_labels = augment_dataset(images, labels , num_images=679)
# %store augmented_images
# %store augmented_labels

print("Augmented DataSet Size :" , len(augmented_images))
assert len(augmented_images) == len(augmented_labels) , "Images and labels of augmented Dataset are not of the same size"
view_random_images(augmented_images, augmented_labels , num_images=10)

In [ ]:
import matplotlib.pyplot as plt
num = np.random.randint(0,len(train_images))
# num = 797
print(num)
img = preprocess_image(images[num])
#Shape of the image
print("Shape of the image :" , img.shape)
print("Label: " , labels[num])
#PLOT 2 IMAGES
plt.subplot(1,2,1)
plt.imshow(images[num])
plt.subplot(1,2,2)
plt.imshow(img , cmap='binary')
plt.show()

In [ ]:
#preprocess the images and store them in place
for i in range(len(images)):
    images[i] = preprocess_image(images[i])
# view some random images
view_random_images(images, labels , num_images=10)

In [ ]:
#preprocess the augmented images and store them in place
for i in range(len(augmented_images)):
    augmented_images[i] = preprocess_image(augmented_images[i])
# view some random images
view_random_images(augmented_images, augmented_labels , num_images=10)

In [ ]:
train_images, train_labels, test_images, test_labels, val_images, val_labels = create_stratified_split(images= images , labels= labels , test_size=0.1, val_size=0.1)
# %store train_images
# %store train_labels
# %store test_images
# %store test_labels
# %store val_images
# %store val_labels

print("Train Set Size :" , len(train_images))
print("Test Set Size :" , len(test_images))
print("Val Set Size :" , len(val_images))

In [ ]:
get_split_stats(train_labels, test_labels, val_labels)

In [ ]:
#all augmented images are used for training
# train_images.extend(augmented_images)
# train_labels.extend(augmented_labels)



In [ ]:
print("Train Set Size :" , len(train_images))
assert len(train_images) == len(train_labels) , "Images and labels of train Dataset are not of the same size"

In [ ]:
from modules.feature_extraction import visualize_HOG
## here we explore the HOG features of a random image
## and how the HOG features change with different parameters
## the default parameters are the ones we chose 
index = np.random.randint(0,len(train_images))
image = train_images[index]
print("Label: " , train_labels[index])
visualize_HOG(image , pixels_per_cell= (16,16) , cells_per_block= (16,16) , orientation= 3)

In [ ]:
index

In [ ]:
# from modules.feature_extraction import extract_features_skimage

# # Extract all features from the whole dataset
# train_features = np.array([])

# for image in train_images:
#     features = extract_features_skimage(image)
#     if train_features.size == 0:
#         train_features = features
#     else:
#         train_features = np.vstack((train_features, features))

In [ ]:
# print(train_features.shape)

In [ ]:
# import pickle
# # Save the train_features in a pickle file
# with open('train_features.pkl', 'wb') as f:
#     pickle.dump(train_features, f)

In [ ]:
import pickle 
# load the train_features from the pickle file
with open('train_features.pkl', 'rb') as f:
    train_features = pickle.load(f)


### Try different values of n_components 
- The extracted features should be experimented to choose the best value for n_components. 

In [ ]:
from modules.feature_extraction import reduce_features

# Reduce the dimensionality of the features
pca3, train_features_reduced3 = reduce_features(train_features, n_components= 0.45)

In [ ]:
# Reduce the dimensionality of the features
pca4, train_features_reduced4 = reduce_features(train_features, n_components= 0.20)

In [ ]:
# Reduce the dimensionality of the features
pca5, train_features_reduced5 = reduce_features(train_features, n_components= 0.10)

In [ ]:
# print all the reduced features shapes
print(train_features_reduced3.shape)
print(train_features_reduced4.shape)
print(train_features_reduced5.shape)

In [ ]:
#extract features from the test images
test_features = np.array([])
for image in test_images:
    features = extract_features_skimage(image)
    if test_features.size == 0:
        test_features = features
    else:
        test_features = np.vstack((test_features, features))

In [ ]:
print("Test_features_shape: ",str(test_features.shape))

In [ ]:
test_features_reduced3 = pca3.transform(test_features)
test_features_reduced4 = pca4.transform(test_features)
test_features_reduced5 = pca5.transform(test_features)

In [ ]:
print("Test_features_reduced_shape: ",str(test_features_reduced3.shape))
print("Test_features_reduced_shape: ",str(test_features_reduced4.shape))
print("Test_features_reduced_shape: ",str(test_features_reduced5.shape))

In [ ]:
#Extracting Digit labels
train_labels_digit = [x[1] for x in train_labels]
test_labels_digit = [x[1] for x in test_labels]

#Converting Labels to NP array
train_labels_digit = np.array(train_labels_digit).astype(int)
test_labels_digit = np.array(test_labels_digit).astype(int)

#printing Label vector shape
print("Train_labels_shape: ",str(train_labels_digit.shape))
print("Test_labels_shape: ",str(test_labels_digit.shape))


In [ ]:
from modules.models import svm_model

SVM_model5 = svm_model(train_features_reduced5, train_labels_digit, test_features_reduced5, test_labels_digit)

In [ ]:
NN_model5 = nn_model(train_features_reduced5, train_labels_digit, test_features_reduced5, test_labels_digit)

In [ ]:
model4 = svm_model(train_features_reduced4, train_labels_digit, test_features_reduced4, test_labels_digit)

In [ ]:
nn_model4 = nn_model(train_features_reduced4, train_labels_digit, test_features_reduced4, test_labels_digit)

In [ ]:
model3 = svm_model(train_features_reduced3, train_labels_digit, test_features_reduced3, test_labels_digit)

In [ ]:
nn_model3 = nn_model(train_features_reduced3, train_labels_digit, test_features_reduced3, test_labels_digit)